[미니 프로젝트]
> 얼굴 인식 기반 AR 필터 적용 프로그램 개발

  - 실시간 웹캠 영상을 이용하여 얼굴을 인식하고, 인식된 얼굴에 가상의 필터  
    이미지(예: 안경, 모자, 수염 등) 를 자연스럽게 덧씌우는 프로그램을 구현

[구현 목표]
 - OpenCV를 활용한 실시간 영상 처리
 - HaarCascade를 활용한 얼굴 검출 알고리즘 이해
 - 알파 채널(투명도) 개념과 이미지 합성 방법 습득
 - c좌표 변환, ROI 설정, 필터 크기 조정 등 실무 이미지 처리 기법 학습

 [구현 필수 요구사항]
1. 기본 구현 (필수)
  1) 웹캠 사용 : cv2.VideoCapture(0)을 사용해 실시간 영상을 불러올 것
  2) 얼굴 검출 : OpenCV의 CascadeClassifier로 얼굴을 실시간 검출할 것
  3) 필터 이미지 적용   안경(또는 모자, 수염 등 자신이 선택한 이미지)의 PNG 이미지를 얼굴 위에 정확히 덧씌울 것
  4) 크기 자동 조절 :   얼굴 크기에 맞게 필터 이미지의 크기를 동적으로 변경할 것
  5) 배경 제거 : 알파 채널이 포함된 PNG 파일을 사용하여 필터 이미지의 투명한 배경을 유지할 것
    > 이미지는 보통 빨강(R), 초록(G), 파랑(B) 이렇게 3개의 색상 채널로 이루어져 있음
    > 여기에 "투명도(불투명도)"를 나타내는 추가 채널을 붙인 것이 바로 알파(Alpha) 채널임
    > 알파 값이 0에 가까울수록 투명, 255에 가까울수록 불투명
    > 예시 : ilter_img = cv2.imread("glasses.png", cv2.IMREAD_UNCHANGED)

즉, 알파 채널이 있는 이미지 = 4채널 이미지 (R, G, B, A)
  6) 화면 출력 : 필터가 적용된 실시간 화면을 cv2.imshow()를 통해 출력할 것
  7) 종료 기능 :ESC 키 누르면 종료되도록 설정할 것


[주요 로직]
1. 필터 이미지 불러오기 (알파 채널 포함 PNG)
2. 얼굴 인식 모델 로딩 (Haar Cascade)
3. 웹캠 연결 및 영상 프레임 반복 처리
    ├── 3-1. 프레임 → 흑백 변환
    ├── 3-2. 얼굴 검출
    ├── 3-3. 얼굴 좌표에 필터 적용
        ├── 필터 크기 조정 (얼굴 크기에 비례)
        ├── 알파 채널 분리 (마스크 생성)
        ├── ROI 설정
        ├── 마스크 기반 합성 처리
4. 결과 영상 출력
5. ESC 입력 시 종료


[주요 이용 함수]

- cv2.imread() 함수는 필터 이미지를 불러올 때 사용하며, 알파 채널(투명도) 을 포함하려면 cv2.IMREAD_UNCHANGED 옵션을 반드시 사용해야 함.
- cv2.CascadeClassifier() 클래스는 얼굴 검출을 위한 모델 로딩에 사용되며, 보통 haarcascade_frontalface_default.xml을 적용함.
- detectMultiScale() 함수는 얼굴을 검출하는 기능을 수행하며, scaleFactor=1.3, minNeighbors=5 등의 파라미터를 기본으로 설정하는 것이 적절함.
- cv2.VideoCapture(0) 함수는 기본 웹캠을 연결하기 위한 함수이며, 연결이 실패했는지 확인하는 조건문을 반드시 포함해야 함.
- cv2.resize() 함수는 필터 이미지를 얼굴 크기에 맞게 조절하기 위해 사용되며, w, h를 기준으로 적절한 비율로 크기 조정이 필요함.
- filter[:, :, 3] 구문은 알파 채널(4번째 채널)을 분리할 때 사용되며, shape[2] == 4일 경우에만 적용 가능하므로 조건문으로 예외 처리를 해야 함.
- cv2.bitwise_not() 함수는 알파 채널을 반전시켜 배경 제거용 마스크(mask_inv)를 생성할 때 사용함.
- ROI = frame[y:y+h, x:x+w]는 얼굴 영역만 잘라내는 방식이며, 필터 이미지가 영상 범위를 벗어나는 경우에 대비한 예외 처리가 필요함.
- cv2.bitwise_and() 함수는 배경 또는 필터 이미지 중 하나에 마스크를 적용해 겹치지 않게 합성하는 데 사용됨.
- cv2.add() 함수는 배경 이미지와 필터 이미지를 결합할 때 사용되며, ROI 영역에 필터를 덧씌우는 최종 단계에서 활용됨.
- cv2.imshow() 함수는 결과를 실시간으로 확인할 수 있도록 영상 출력을 담당하며, 창 이름은 "Face Filter" 등으로 지정함.
- cv2.waitKey(1) 함수는 키 입력을 대기하는 역할을 하며, & 0xFF == 27 조건을 통해 ESC 키가 눌렸을 때 프로그램이 종료되도록 설정해야 함.
- cv2.destroyAllWindows() 함수는 종료 시 모든 OpenCV 창을 닫는 역할을 하며, cap.release()와 함께 반드시 포함시켜야 함.

In [1]:
# 라이브러리 불러오기
import cv2
import numpy as np

In [2]:
# 필터 이미지 불러오기 (알파 채널 포함)
filter_img = cv2.imread("./data/glasses.png", cv2.IMREAD_UNCHANGED)  # PNG 파일, 4채널
if filter_img is None:
    print("필터 이미지를 불러올 수 없습니다.")
    exit()

# 얼굴 인식 모델 로딩
cascade_path = cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
face_cascade = cv2.CascadeClassifier(cascade_path)

# 웹캠 연결
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("웹캠을 열 수 없습니다.")
    exit()

print("Face Filter 실행! ESC 키로 종료합니다.")

while True:
    ret, frame = cap.read()
    if not ret:
        print("프레임을 읽을 수 없습니다.")
        break

    # 흑백 변환
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # 얼굴 검출
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)

    for (x, y, w, h) in faces:
        #필터 크기 조정
        filter_resized = cv2.resize(filter_img, (w, int(filter_img.shape[0] * (w / filter_img.shape[1]))))
        fh, fw = filter_resized.shape[:2]

        # 얼굴 위에 필터 위치 설정 (예: 눈 위쪽)
        y1 = y
        y2 = y + fh
        x1 = x
        x2 = x + fw

        # 영상 범위를 벗어나면 잘라내기
        if y2 > frame.shape[0]:
            y2 = frame.shape[0]
            filter_resized = filter_resized[:y2 - y1, :, :]
        if x2 > frame.shape[1]:
            x2 = frame.shape[1]
            filter_resized = filter_resized[:, :x2 - x1, :]

        # 알파 채널 분리
        if filter_resized.shape[2] == 4:
            alpha = filter_resized[:, :, 3]  # 알파 채널
            alpha = cv2.merge([alpha, alpha, alpha])  # 3채널로 변환
            alpha = alpha / 255.0  # 0~1로 정규화

            # ROI 설정
            roi = frame[y1:y2, x1:x2]

            # 마스크 기반 합성
            foreground = cv2.multiply(alpha, filter_resized[:, :, :3].astype(float))
            background = cv2.multiply(1.0 - alpha, roi.astype(float))
            out_image = cv2.add(foreground, background)

            frame[y1:y2, x1:x2] = out_image.astype(np.uint8)

    # 결과 영상 출력
    cv2.imshow("Face Filter", frame)

    # ESC 키 종료
    key = cv2.waitKey(1) & 0xFF
    if key == 27:
        break

# 자원 해제
cap.release()
cv2.destroyAllWindows()


Face Filter 실행! ESC 키로 종료합니다.
